In [8]:
from urllib.request import urlopen
target_url = \
    'https://www.boatrace.jp/owpc/pc/race/odds3t?rno=12&jcd=02&hd=20200511'
html_content = urlopen(target_url).read()
print(type(html_content))

<class 'bytes'>


In [9]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [16]:
target_table_selector = \
    'body > main > div > div > div > '\
    'div.contentsFrame1_inner > div:nth-child(6) > table'
odds_table = soup.select_one(target_table_selector)
print(type(odds_table))

<class 'bs4.element.Tag'>


In [20]:
# tbodyの指定
odds_table_elements = odds_table.select_one('tbody')

# trを指定しリストとして格納する
row_list = odds_table_elements.select('tr')
print(type(row_list))
print(len(row_list))

<class 'bs4.element.ResultSet'>
20


In [33]:
# 各行に対して行う処理
def getoddsPoint2floatlist(odds_tr):
    # odds値が格納された部分のhtmlのリストを取得
    html_list = odds_tr.select('td.oddsPoint')
#     print(html_list[0])
    # example output:
    # <td class="oddsPoint">47.2</td>
    # textをつかうことでタグで囲まれた要素のみを抜き出せる
    text_list = list(map(lambda x: x.text, html_list))
    # print(text_list)
    # example output:
    # ['47.2', '60.3', '588.7', '52.8', '66.0', '248.7']
#     print(text_list)
    # oddsは小数値なのでflot型へキャスト
    float_list = list(map(
        lambda x: float(x), text_list))
    return float_list
getoddsPoint2floatlist(row_list[0])

[47.2, 60.3, 588.7, 52.8, 66.0, 248.7]

In [35]:
odds_matrix = list(map(
    lambda x: getoddsPoint2floatlist(x),
    row_list
))

print(odds_matrix)

[[47.2, 60.3, 588.7, 52.8, 66.0, 248.7], [14.7, 13.3, 994.9, 361.6, 363.8, 1276.0], [12.0, 11.1, 747.7, 67.1, 137.8, 503.6], [26.7, 26.6, 1155.0, 96.5, 123.7, 414.5], [157.0, 188.8, 566.8, 50.4, 64.3, 241.5], [242.2, 215.7, 660.5, 261.5, 314.5, 1037.0], [237.5, 190.8, 561.6, 36.4, 66.8, 183.4], [403.5, 281.1, 926.8, 49.2, 73.1, 183.6], [35.0, 25.4, 1276.0, 750.0, 930.3, 2462.0], [219.2, 152.2, 959.6, 517.5, 799.1, 1950.0], [59.6, 23.6, 963.4, 650.0, 1139.0, 1779.0], [89.4, 38.4, 1433.0, 639.7, 1237.0, 2321.0], [34.6, 23.8, 1019.0, 63.9, 119.7, 387.5], [212.5, 143.8, 752.3, 36.9, 64.1, 174.3], [76.3, 30.5, 1231.0, 270.8, 452.2, 952.1], [79.6, 35.8, 1614.0, 44.9, 84.1, 244.4], [83.7, 90.6, 2031.0, 110.1, 171.1, 391.8], [356.3, 308.5, 1552.0, 63.2, 103.9, 201.7], [159.7, 77.7, 1408.0, 326.7, 560.3, 1346.0], [136.0, 69.0, 1562.0, 71.4, 148.1, 285.7]]


In [39]:
import numpy as np
# numpy array化
odds_matrix = np.array(odds_matrix)
# 転置を取り，つなげてリスト化
odds_list = list(odds_matrix.T.reshape(-1))

# 辞書で格納する
three_rentan_odds_dict = {}
for fst in range(1, 7):
    if fst not in three_rentan_odds_dict.keys():
        three_rentan_odds_dict[str(fst)] = {}
    for snd in range(1, 7):
        if snd != fst:
            if snd not in three_rentan_odds_dict[str(fst)].keys():
                three_rentan_odds_dict[str(fst)][str(snd)] = {}
            for trd in range(1, 7):
                if trd != fst and trd != snd:
                    three_rentan_odds_dict[str(fst)][str(snd)][str(trd)] = \
                        odds_list.pop(0)

In [40]:
print('sample1:', three_rentan_odds_dict['1']['2']['3'])
print('sample2:', three_rentan_odds_dict['6']['5']['4'])

sample1: 47.2
sample2: 285.7
